# Part 2 - Generators

## Intro

In Python, generators are an abstraction that allows for creating iterators that have some very useful properties for dealing with large datasets:
- **lazy evaluation**: generators don't require all data to be held in memory. 
- **stateful iterator**: generators implicitly remember "where" they are in the sequence they generate.
- **`yield`**: generators have a very simple "drop-in" syntax; using the `yield` or `yield from` keywords will cause whatever function using them to become a generator, and implements the iterator protocol. 

Let's say you wanted to generate an arbitrary number of powers of two. Here are three ways you could do it:

In [7]:
# This way takes up linear memory; unless we need all these powers at the same time, this is wasteful.
def generate_pow2(n):
    pows = []
    for i in range(n):
        pows.append(2**i)
    return pows
print(generate_pow2(10))

# This iterator only uses O(c) memory by contrast, but it's a lot of code for such a simple task.
class Pow2Iterator:
    def __init__(self, n):
        self.n = n
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i == self.n:
            raise StopIteration
        val = 2**self.i
        self.i += 1
        return val
print(list(Pow2Iterator(10)))

# Finally, this implementation uses a generator; it also uses O(c) memory, and is only a few lines of code
def gen_pow2(n):
    for i in range(n):
        yield 2**i
print(list(gen_pow2(10)))

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]


## Nesting generators

Generators can be used within other generators, which can be made quite concise with `yield from`. For instance, let's say we wanted to flatten an arbitrarily-deep nested list (which can contain empty sublists). Doing this with nested generators and recursion is very straightforward. Recursion works well here because of the nesting structure. And because the generators are stateful, they "remember" where they left off after each yield so we don't need to keep track of a stack or explicit iteration variables. 

In [9]:
nested_list = [[1,2,3], 4, [5,[6,7]], [[[8]]], [9], [10,[11,[12,[13]]]]]

def list_flattener(arr):
    for item in arr:
        if isinstance(item, list):
            yield from list_flattener(item)
        else:
            yield item

list(list_flattener(nested_list))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

Note though that even though generators are very memory efficient due to lazy evaluation, this approach is still O(n) for memory - each recursion triggers a new stack frame, and we recurse each time we go to the next nested list. The worst case looks like `[[[[[[[[[[[ ... [1] ...]]]]]]]]]]]`. 

## `send()`

`send()` allows you to send data back to the generator. Generators must be structured specifically to make use of this - not every generator can be interacted with via `send()`. The trick here is that `yield` can also be used to recieve data from `send()`, in addition to returning data to the callee. 

In [30]:
def sendable_pow2(n):
    i = 0
    while i < n:
        # This line is doing 2 things:
        # - actually yielding the value to the caller
        # - yield can also recieve a value from send() when used
        #   outside the generator.
        exponent = yield 2**i
        
        # So here, if we recieved a value from send(), save it
        # as i.
        if exponent is not None:
            i = exponent
        else:
            # Otherwise, just increment i as normal. 
            i += 1
            
# This generator can still be used as normal
list(sendable_pow2(10))

# But now we can do this:
it = sendable_pow2(100)
print(next(it))     # 2^0 = 1
print(next(it))     # 2^1 = 2
print(it.send(10))  # 2^10 = 1024
print(next(it))     # 2^11 = 2048
print(it.send(1))
print(next(it))
print(next(it))
print(next(it))

1
2
1024
2048
2
4
8
16
